# Model Training

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestRegressor,
                              AdaBoostRegressor ,
                                GradientBoostingRegressor,
                                VotingRegressor,
                                StackingRegressor)

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error , r2_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler # for num_cols
from sklearn.preprocessing import OneHotEncoder # for cat_cols
from sklearn.compose import ColumnTransformer # to transform features
from sklearn.model_selection import train_test_split

import pickle
import os

In [2]:
df = pd.read_csv('data/laptop_data_cleaned.csv')
df.drop_duplicates(inplace=True)

In [3]:
df

,Company,TypeName,Ram,Weight,Price,TouchScreen,Ips,Ppi,Cpu_brand,HDD,SSD,Gpu_brand,Os
0,Apple,Ultrabook,8,1.37,11.175755,0,1,226.983005,Intel Core i5,0,128,Intel,Mac
1,Apple,Ultrabook,8,1.34,10.776777,0,0,127.677940,Intel Core i5,0,0,Intel,Mac
2,HP,Notebook,8,1.86,10.329931,0,0,141.211998,Intel Core i5,0,256,Intel,Others
3,Apple,Ultrabook,16,1.83,11.814476,0,1,220.534624,Intel Core i7,0,512,AMD,Mac
4,Apple,Ultrabook,8,1.37,11.473101,0,1,226.983005,Intel Core i5,0,256,Intel,Mac
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,Asus,Notebook,4,2.20,10.555257,0,0,100.454670,Intel Core i7,500,0,Nvidia,Windows
1269,Lenovo,2 in 1 Convertible,4,1.80,10.433899,1,1,157.350512,Intel Core i7,0,128,Intel,Windows
1270,Lenovo,2 in 1 Convertible,16,1.30,11.288115,1,1,276.053530,Intel Core i7,0,512,Intel,Windows
1271,Lenovo,Notebook,2,1.50,9.409283,0,0,111.935204,Other Intel Processor,0,0,Intel,Windows


In [4]:
df.describe()

,Ram,Weight,Price,TouchScreen,Ips,Ppi,HDD,SSD
count,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000,1272.000000
mean,8.451258,2.041761,10.828035,0.147013,0.279874,146.936126,414.040881,186.298742
std,5.099249,0.669088,0.619774,0.354258,0.449114,42.940459,518.128204,186.597777
min,2.000000,0.690000,9.134616,0.000000,0.000000,90.583402,0.000000,0.000000
25%,4.000000,1.500000,10.387379,0.000000,0.000000,127.335675,0.000000,0.000000
50%,8.000000,2.040000,10.872255,0.000000,0.000000,141.211998,0.000000,256.000000
75%,8.000000,2.310000,11.287447,0.000000,1.000000,157.350512,1000.000000,256.000000
max,64.000000,4.700000,12.691441,1.000000,1.000000,352.465147,2000.000000,1024.000000


In [5]:
X = df.drop(columns=['Price'])
y = df['Price']

In [56]:
cat_cols = [features for features in X.columns if X[features].dtypes == 'O']
num_cols = [features for features in X.columns if X[features].dtypes != 'O']
len(num_cols) , len(cat_cols)

(7, 5)

In [57]:
cat_pipeline = Pipeline(
    steps=[
        ('ohe',OneHotEncoder(sparse=False, # this will return me numpy array by the first time
        drop='first'# This will drop first column
              )),
        ('imputer',SimpleImputer())
    ]
)

In [58]:
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer()),
        ('scaler',StandardScaler()),
    ]
)

In [59]:
preprocessor_obj = ColumnTransformer(
    transformers=[
        ('cat_col_pipeline',cat_pipeline,cat_cols),
        ('num_col_pipeline',num_pipeline,num_cols)
    ],remainder='passthrough'
)
X = preprocessor_obj.fit_transform(X)

c:\Users\Gyanprakash\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [60]:

X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.2, random_state=1)

X_train.shape , X_test.shape , y_train.shape ,y_test.shape


((1017, 38), (255, 38), (1017,), (255,))

In [61]:
X_train.shape


(1017, 38)

In [12]:
dt = DecisionTreeRegressor()
dt.fit(X_train,y_train)

DecisionTreeRegressor()

In [13]:
y_pred = dt.predict(X_test)

In [14]:
r2_score(y_test,y_pred)

0.8057911295967265

In [15]:
models = {
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor(),
}


In [16]:
params = {
    "Random Forest": {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        # 'max_features': ['sqrt', 'log2', None],
        'n_estimators': [8, 16, 32, 64, 128, 256]
    },
    "Gradient Boosting": {
        'loss': ['squared_error', 'huber', 'absolute_error', 'quantile'],
        'learning_rate': [0.1, 0.01, 0.05, 0.001],
        # 'subsample': [0.6, 0.7, 0.75, 0.8, 0.85, 0.9],
        'criterion': ['squared_error', 'friedman_mse'],
        # 'max_features': ['auto', 'sqrt', 'log2'],
        'n_estimators': [8, 16, 32, 64, 128, 256]
    },
    "XGBRegressor": {
        'learning_rate': [0.1, 0.01, 0.05, 0.001],
        'n_estimators': [8, 16, 32, 64, 128, 256]
    },
    "CatBoosting Regressor": {
        'depth': [6, 8, 10],
        'learning_rate': [0.01, 0.05, 0.1],
        'iterations': [30, 50, 100]
    },
    "AdaBoost Regressor": {
        'learning_rate': [0.1, 0.01, 0.5, 0.001],
        # 'loss': ['linear', 'square', 'exponential'],
        'n_estimators': [8, 16, 32, 64, 128, 256]
    }
}


In [17]:
len(params.keys())

5

In [18]:
def error_Accuracy(true,pred):
    r2_accuracy = r2_score(y_true=true,y_pred=pred)
    mse = mean_squared_error(y_true=true,y_pred=pred)
    mae = mean_absolute_error(y_true=true,y_pred=pred)
    return r2_accuracy,mse,mae

In [19]:

class ModelTraining:
    def __init__(self):
        self.model_list = []
        self.accuracy_list = []
        self.best_params_dict = {}

    def train_model(self, models, params, X_train, y_train, X_test, y_test):
        for i, model_name in enumerate(models.keys()):
            model = models[model_name]
            param = params[model_name]

            grid_search_cv = GridSearchCV(estimator=model, param_grid=param, cv=5)
            grid_search_cv.fit(X_train, y_train)

            best_params = grid_search_cv.best_params_
            model.set_params(**best_params)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            r2_accuracy, mse, mae = error_Accuracy(true=y_test, pred=y_pred)

            print(model_name)
            self.model_list.append(model_name)

            print("- Mean Absolute Error: {:.4f}".format(mae))
            print("- Mean Squared Error: {:.4f}".format(mse))
            print("- R2 Score: {:.4f}".format(r2_accuracy))
            print('\n')

            self.best_params_dict[model_name] = best_params

            self.accuracy_list.append(r2_accuracy)

    def return_accuracy_list(self):
        return self.accuracy_list
    
    def best_params(self):
        return self.best_params_dict


In [20]:
train_model = ModelTraining()

In [21]:
train_model.train_model(models=models,params=params,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)

Random Forest
- Mean Absolute Error: 0.1537
- Mean Squared Error: 0.0402
- R2 Score: 0.8889


Gradient Boosting
- Mean Absolute Error: 0.1600
- Mean Squared Error: 0.0411
- R2 Score: 0.8864


XGBRegressor
- Mean Absolute Error: 0.1396
- Mean Squared Error: 0.0346
- R2 Score: 0.9044


CatBoosting Regressor
- Mean Absolute Error: 0.1583
- Mean Squared Error: 0.0427
- R2 Score: 0.8818


AdaBoost Regressor
- Mean Absolute Error: 0.2074
- Mean Squared Error: 0.0669
- R2 Score: 0.8151




In [22]:
train_model.best_params_dict

{'Random Forest': {'criterion': 'squared_error', 'n_estimators': 64},
 'Gradient Boosting': {'criterion': 'friedman_mse',
  'learning_rate': 0.1,
  'loss': 'huber',
  'n_estimators': 256},
 'XGBRegressor': {'learning_rate': 0.1, 'n_estimators': 256},
 'CatBoosting Regressor': {'depth': 10,
  'iterations': 100,
  'learning_rate': 0.1},
 'AdaBoost Regressor': {'learning_rate': 0.5, 'n_estimators': 128}}

In [23]:
train_model.accuracy_list

[0.8888722465503275,
 0.8864045035992347,
 0.90441338344959,
 0.881847048029996,
 0.8150973848035081]

In [24]:
model_accuracy=pd.DataFrame(list(zip(list(models.keys()),train_model.accuracy_list)),columns=['model','accuracy'])
model_accuracy

,model,accuracy
0,Random Forest,0.888872
1,Gradient Boosting,0.886405
2,XGBRegressor,0.904413
3,CatBoosting Regressor,0.881847
4,AdaBoost Regressor,0.815097


In [25]:
best_models = model_accuracy.sort_values(by='accuracy',ascending=False).head(4)
best_models

,model,accuracy
2,XGBRegressor,0.904413
0,Random Forest,0.888872
1,Gradient Boosting,0.886405
3,CatBoosting Regressor,0.881847


In [26]:
best_models

,model,accuracy
2,XGBRegressor,0.904413
0,Random Forest,0.888872
1,Gradient Boosting,0.886405
3,CatBoosting Regressor,0.881847


## Voting Regressor

In [63]:
train_model.best_params_dict

{'Random Forest': {'criterion': 'squared_error', 'n_estimators': 64},
 'Gradient Boosting': {'criterion': 'friedman_mse',
  'learning_rate': 0.1,
  'loss': 'huber',
  'n_estimators': 256},
 'XGBRegressor': {'learning_rate': 0.1, 'n_estimators': 256},
 'CatBoosting Regressor': {'depth': 10,
  'iterations': 100,
  'learning_rate': 0.1},
 'AdaBoost Regressor': {'learning_rate': 0.5, 'n_estimators': 128}}

In [65]:
train_model.best_params_dict['Random Forest']

{'criterion': 'squared_error', 'n_estimators': 64}

In [67]:
RandomForest =RandomForestRegressor().set_params(**train_model.best_params_dict['Random Forest'])
GradientBoost = GradientBoostingRegressor().set_params(**train_model.best_params_dict['Gradient Boosting'])
Xgboost = XGBRegressor().set_params(**train_model.best_params_dict['XGBRegressor'])
Catboost = CatBoostRegressor().set_params(**train_model.best_params_dict['CatBoosting Regressor'])


In [69]:
votingRegressor = VotingRegressor(
    estimators=[
        ('RandomForest',RandomForest),
        ('GradientBoost',GradientBoost),
        ('Xgboost',Xgboost),
        ('Catboost',Catboost)
    ]
)

In [70]:
votingRegressor.fit(X_train,y_train)

0:	learn: 0.5852870	total: 7.44ms	remaining: 737ms
1:	learn: 0.5483074	total: 16.1ms	remaining: 788ms
2:	learn: 0.5137807	total: 23.3ms	remaining: 753ms
3:	learn: 0.4843408	total: 32ms	remaining: 768ms
4:	learn: 0.4572128	total: 39.4ms	remaining: 748ms
5:	learn: 0.4338705	total: 40.8ms	remaining: 639ms
6:	learn: 0.4091679	total: 48.6ms	remaining: 645ms
7:	learn: 0.3877900	total: 50.7ms	remaining: 583ms
8:	learn: 0.3700159	total: 57.6ms	remaining: 583ms
9:	learn: 0.3533477	total: 66.3ms	remaining: 596ms
10:	learn: 0.3380532	total: 73.5ms	remaining: 594ms
11:	learn: 0.3258126	total: 81.9ms	remaining: 601ms
12:	learn: 0.3130784	total: 89.1ms	remaining: 597ms
13:	learn: 0.3010350	total: 97.5ms	remaining: 599ms
14:	learn: 0.2916865	total: 105ms	remaining: 593ms
15:	learn: 0.2838042	total: 106ms	remaining: 557ms
16:	learn: 0.2754030	total: 114ms	remaining: 557ms
17:	learn: 0.2687499	total: 122ms	remaining: 554ms
18:	learn: 0.2627074	total: 130ms	remaining: 555ms
19:	learn: 0.2566275	total: 1

VotingRegressor(estimators=[('RandomForest',
                             RandomForestRegressor(n_estimators=64)),
                            ('GradientBoost',
                             GradientBoostingRegressor(loss='huber',
                                                       n_estimators=256)),
                            ('Xgboost',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval...
                                          learning_rate=0.1, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=256, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None, ...)),
                            ('Catboost',
                             <catboost.core.CatBoostRegressor object at 0x00000213446C5590>)])

In [72]:
y_pred_voting_regressor = votingRegressor.predict(X_test)

In [73]:
r2_accuracy, mse, mae = error_Accuracy(true=y_test, pred=y_pred_voting_regressor)
print("- Mean Absolute Error: {:.4f}".format(mae))
print("- Mean Squared Error: {:.4f}".format(mse))
print("- R2 Score: {:.4f}".format(r2_accuracy))

- Mean Absolute Error: 0.1450
- Mean Squared Error: 0.0346
- R2 Score: 0.9044


# Finally Voting Regressor is the best Algo For my MODEL

## Pickling Entire Model

In [33]:
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.2, random_state=1)

X_train.shape , X_test.shape , y_train.shape ,y_test.shape


((1017, 12), (255, 12), (1017,), (255,))

In [34]:
y_train

208     11.254190
1015    11.073937
754     10.720680
76      10.227465
228     10.767905
          ...    
715     10.729081
905     11.624777
1097    11.352695
235     10.776844
1062    10.301710
Name: Price, Length: 1017, dtype: float64

In [35]:
preprocess_model = Pipeline(
    steps=[
   ( 'preprocessor_obj',preprocessor_obj),
   ('votingRegressor',votingRegressor)
    ]
)
preprocess_model.fit_transform(X_train,y_train)


c:\Users\Gyanprakash\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0:	learn: 0.4550098	total: 1.23ms	remaining: 312ms
1:	learn: 0.3455767	total: 2.34ms	remaining: 297ms
2:	learn: 0.2889211	total: 3.4ms	remaining: 287ms
3:	learn: 0.2644756	total: 4.47ms	remaining: 282ms
4:	learn: 0.2525025	total: 5.59ms	remaining: 281ms
5:	learn: 0.2404295	total: 6.65ms	remaining: 277ms
6:	learn: 0.2343844	total: 7.73ms	remaining: 275ms
7:	learn: 0.2312554	total: 8.73ms	remaining: 271ms
8:	learn: 0.2268876	total: 9.8ms	remaining: 269ms
9:	learn: 0.2209449	total: 10.8ms	remaining: 267ms
10:	learn: 0.2170065	total: 12.1ms	remaining: 269ms
11:	learn: 0.2142584	total: 13.5ms	remaining: 275ms
12:	learn: 0.2086405	total: 14.6ms	remaining: 273ms
13:	learn: 0.2054647	total: 15.6ms	remaining: 269ms
14:	learn: 0.2019314	total: 16.7ms	remaining: 268ms
15:	learn: 0.1986108	total: 17.8ms	remaining: 267ms
16:	learn: 0.1943821	total: 18.8ms	remaining: 265ms
17:	learn: 0.1919346	total: 19.9ms	remaining: 262ms
18:	learn: 0.1898856	total: 20.9ms	remaining: 260ms
19:	learn: 0.1879048	tot

array([[11.28851295, 11.26985113, 11.26846313, 11.26580251],
       [11.05556867, 10.97259609, 11.03379059, 11.04266196],
       [10.67549127, 10.52961076, 10.71079922, 10.75841943],
       ...,
       [11.2450198 , 11.16702566, 11.32951164, 11.29653152],
       [10.86411193, 10.79795091, 10.80856609, 10.79524618],
       [10.38266327, 10.47556012, 10.33291721, 10.336334  ]])

In [36]:
os.makedirs(name='model',exist_ok=True)
pickle.dump(df,open(file='model/df.pkl',mode='wb'))
# pickle.dump(votingRegressor,open(file='model/model.pkl',mode='wb'))
# pickle.dump(preprocessor_obj,open(file='model/preprocessor.pkl',mode='wb'))
pickle.dump(preprocess_model,open(file='model/preprocess_model.pkl',mode='wb'))

In [37]:
df

,Company,TypeName,Ram,Weight,Price,TouchScreen,Ips,Ppi,Cpu_brand,HDD,SSD,Gpu_brand,Os
0,Apple,Ultrabook,8,1.37,11.175755,0,1,226.983005,Intel Core i5,0,128,Intel,Mac
1,Apple,Ultrabook,8,1.34,10.776777,0,0,127.677940,Intel Core i5,0,0,Intel,Mac
2,HP,Notebook,8,1.86,10.329931,0,0,141.211998,Intel Core i5,0,256,Intel,Others
3,Apple,Ultrabook,16,1.83,11.814476,0,1,220.534624,Intel Core i7,0,512,AMD,Mac
4,Apple,Ultrabook,8,1.37,11.473101,0,1,226.983005,Intel Core i5,0,256,Intel,Mac
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,Asus,Notebook,4,2.20,10.555257,0,0,100.454670,Intel Core i7,500,0,Nvidia,Windows
1269,Lenovo,2 in 1 Convertible,4,1.80,10.433899,1,1,157.350512,Intel Core i7,0,128,Intel,Windows
1270,Lenovo,2 in 1 Convertible,16,1.30,11.288115,1,1,276.053530,Intel Core i7,0,512,Intel,Windows
1271,Lenovo,Notebook,2,1.50,9.409283,0,0,111.935204,Other Intel Processor,0,0,Intel,Windows
